In [1]:
from tensorflow.keras.models import load_model

model = load_model("plant_disease_model.keras")

In [2]:
train_dir = "C:/data/New Plant Diseases Dataset/New Plant Diseases Dataset(Augmented)/train"

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224,224)
BATCH_SIZE = 32
EPOCHS = 5
IN_LAST_LAYER = 10
SEED = 1337

# Training data generator with light augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode='reflect'
)


# Create data generators
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

Found 70295 images belonging to 38 classes.


In [3]:
CLASS_NAMES = list(train_gen.class_indices.keys())
print(CLASS_NAMES[:5])

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy']


In [4]:
import json
with open("classes.json","w") as f:
    json.dump(CLASS_NAMES,f)

In [5]:
CLASS_NAMES

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_

In [6]:
import numpy as np
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

IMG_SIZE = (224,224)

def predict_disease(img_path):
    # open image
    img = Image.open(img_path).convert("RGB")
    img = img.resize(IMG_SIZE)

    # to numpy
    img_array = np.array(img, dtype=np.float32)

    # 🔴 MobileNet preprocessing (VERY IMPORTANT)
    img_array = preprocess_input(img_array)

    # batch dimension
    img_array = np.expand_dims(img_array, axis=0)

    # predict
    preds = model.predict(img_array, verbose=0)[0]

    idx = np.argmax(preds)
    disease = CLASS_NAMES[idx]
    confidence = float(preds[idx]) * 100

    return disease, confidence

In [7]:
disease, conf = predict_disease("C:\\data\\New Plant Diseases Dataset\\test\\test\\TomatoEarlyBlight2.JPG")
print("Disease:", disease)
print("Confidence:", f"{conf:.2f}%")

Disease: Tomato___Early_blight
Confidence: 98.20%


In [8]:
disease, conf = predict_disease("test.JPG")
print("Disease:", disease)
print("Confidence:", f"{conf:.2f}%")

Disease: Tomato___Tomato_Yellow_Leaf_Curl_Virus
Confidence: 100.00%


In [ ]:
import json
json.dump(train_gen.class_indices, open("models/class_indices.json","w"))